In [1]:
# Imports
import os
import json
from openai import AzureOpenAI
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery, QueryType, QueryCaptionType, QueryAnswerType
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

# Environment setup
load_dotenv()
service_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT") 
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME") 
deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
key = os.getenv("AZURE_SEARCH_KEY") 
model = "ada002"
language = "python"
verbose = False #Set to true to see more output information

#Initialize AzureOpenAI client
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_KEY'],  
  api_version = "2023-12-01-preview"
  )

messages=[]

In [2]:
# Function to generate embeddings for title and content fields, also used for query embeddings
def generate_embeddings(text):
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# Gets the relevant documents from Azure Cognitive Search.
def get_from_RAG(query: str) -> list[str]:    
    search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
    vector_query = VectorizedQuery(vector= generate_embeddings(query), k_nearest_neighbors=3, fields="contentVector")
    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["title", "content", "categories","doctype"],
        query_type=QueryType.SEMANTIC, 
        semantic_configuration_name='my-semantic-config', 
        query_caption=QueryCaptionType.EXTRACTIVE, 
        query_answer=QueryAnswerType.EXTRACTIVE,
        top=3
    )
    semantic_answers = results.get_answers()
    if verbose:
        for answer in semantic_answers:
            print(f"Semantic Answer Score: {answer.score}")
            print(f"Semantic Answer: {answer.text}\n")            
        
    rag_results=[]
    for result in results:
        if verbose:            
            print(f"Title: {result['title']}")
            print(f"Reranker Score: {result['@search.reranker_score']}")
            print(f"Content: {result['content']}")
            print(f"Category: {result['categories']}")
            captions = result["@search.captions"]
            if captions:
                caption = captions[0]
                print(f"Caption: {caption.text}\n")
        rag_results.append(result['content'])                
                
    return rag_results

result= get_from_RAG("Find library to cut material vertical for SE-01 machine")
# print(result)

In [12]:
# system message
system_message = f'''
You are a an expert in generating code using exclusively the {language} language.
Based on the user request you will generate the code for the user following the classic {language} conventions and commenting the code so that it will be easily readable favoring semplicity over complexity.
The generated code should be syntactically correct and should be able to run without errors and the it should include the following features:
- All the required imports should be included and listed at the top of the file.
- The code should be executed via a main function.
- Do not generate additional ouput comments other than the inline with the code.
- Do not generate additional code other than the one requested by the user (example code to initialize the machine, etc.).
- Do not import additional libraries other than the ones required by the code.

In order to generate compliant code you might need to use external libraries, use function provided in the prompt to request detailed information about them.

'''

In [13]:
function_extraction_rules = f'''
The following rules should be used to identify the functions to use from the external libraries:
-Use a specific external function only if:
    - The function name seems to be designed for the specific use case (e.g. blow_air() in case the code should blow air)
    - The optional function comments or related description seems to match the functionality.
    - Associated function comments or related documentation seems to describe the required functionality (e.g. turn_on_machine() is associated with a comment or documentaion that says "This function turns on the machine") despite the name of the function itself seems to disagree:
        Examples:
        - the name of the function is: turn_on_system() but the comment says "turns on the machine" in this case the function should be used when the code should turn on the machine.
        - the name of the function is: HSI(sec) and the comment says "It heats the welding head for the specified amount of seconds" in this case the function should be used when the code is supposed to heat a soldering head.
    - Favor the description of the function over the name of the function, to decide if the function should be used or not.
    - The documentation indicates that the function can be used for the required functionality:
        Examples of when it should NOT be use:
        - Documentation says "This function can be used to turn on the machine (deprecated)" in this case the function should be used because deprecated.
        - Documentation indicates other better alternatives like "This function can be used to turn on the machine (note: use turn_on_machine2() instead)" in this case the suggested function turn_on_machine2() should be picked.

-Do not use the external library functions if:
    - the name of the function seems to be designed for other specific use cases (e.g. turn_on_machine() seems to not be designed to blow air)    
if it appears to have external functions that overlaps in functionality (e.g. blow_air_for(seconds) and blow_air_on() and blow_air_off()) favor the one that result in lesser generated code for the requested functionality.
if none of the external libraries offer a function for the required functionality, insert a comment similar to the following one:

    # NOTE: no external function found to <replace with name the operation to achieve (e.g. 'blow air')>
    <insert a comment with a pseudo library function call, possibly using a meaningful name (e.g. turn_on_machine in case required functionality is to turn on the machine instead of foo() or xxx()) including required parameters that can be used to achieve the required functionality>
    
    the comment shold be placed before the pseudo library function call and should be indented with the same indentation of the pseudo library function call.
    the pseudo library function call should be commented out.

'''

Theses are the user commands, edit and run them to see the different output considering that output also depends on code implemented into function (to be replaced by RAG)

In [14]:

# Prompt #0 easy cut
# user_command = f'''
# Generate a program to control a machine model SE-01 via a programmable logic controller (PLC) to cut a material vertically from the point (13,66) to the point (1366,2703) following a zigzag pattern.
# '''


# Prompt #1 cut
# user_command = f'''
# Generate a program to control a machine model SE-01 via a programmable logic controller (PLC) to cut a material, in a way that reminds a circle following a zigzag pattern from the point (10,10) to the point (100,100).
# '''

# Prompt #2 soldering
# user_command = f'''
# Generate a program to control a machine model SE-01 to solder a material vertically following a zigzag pattern from the point (10,10) to the point (100,100).
# '''

# Prompt #3 cutting and soldering
user_command = f'''
Generate a program to control a machine model SE-01 via a programmable logic controller (PLC) to execute the following operations:

1. cut the material, vertically from the point (10,10) to the point (100,100).
2. create a round cut where point (123,456) is the center and 78 is the radius of the circle.
3. solder the material, horizontally from the point (65,18) to the point (178,481).
4. create a circular welding of the material from center (742,95) with a radius of 12 and border follows a zigzag pattern.
'''

In [15]:
functions = [
   {
      "name":"rag_search_library",
      "description":"Find the library to use for a specific operation (e.g. cut, solder, etc.) for a specific machine (e.g. BK-964, etc.)",
      "parameters": {
         "type":"object",
         "properties":{
            "machineKind":{
               "type":"string",               
               "description": "The name of the machine to use (e.g. ZH-965-AZ, etc.)"
            },  
            "operation":{
               "type":"string",               
               "description": "the kind of operation requested (e.g. cutting, blowing, etc.)"
            },
            "direction":{
               "type":"string",               
               "description": "The kind of operation required (e.g. vertical, horizontal, etc.)"
            },       
         },
         "required":["machineKind","operation","direction"]
      },
   }
]

In [16]:
# Set to true to see RAG output information
verbose = False 

# Handle RAG search
def rag_search_library(machineKind, operation, direction):      
    query=  f"Find library for {operation} material {direction} for {machineKind} machine"
    
    print(f"RAG Query: {query}\n")
    rag_result= get_from_RAG(query)
    return "".join(rag_result)    

# Load functions in a dictionary for later use
available_functions = {
                "rag_search_library": rag_search_library
        }

#Test...
#rag_search_library("SE-01", "blow", "vertical")

In [17]:

# Load info required by the model    
def load_info(response_message):
    function_name = response_message.function_call.name
    if function_name:
        print(f"\nModel wants to call a function '{function_name}'...") 
        function_to_call = available_functions[function_name] 

        function_args = json.loads(response_message.function_call.arguments)
        function_response = function_to_call(**function_args)        
        print(f"Function '{function_name}' returned: {function_response}\n")
        # Append function info to messages
        messages.append( # adding assistant response to messages
            {
                "role": response_message.role,
                "function_call": {
                    "name": function_name,
                    "arguments": response_message.function_call.arguments,
                },
                "content": None
            }
        )
        messages.append( # adding function response to messages
            {
                "role": "function",
                "name": function_name,
                "content":function_response,
            }
       ) 

# Asks the model to generate the code
def generate_code():
    openai_response = client.chat.completions.create(
        model=deployment,    
        messages = messages,
        temperature=0.3,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        functions=functions,
        function_call="auto"
    )
    return openai_response.choices[0].message

In [18]:
messages.append({'role': 'system', 'content': system_message})
messages.append({'role': 'user', 'content': function_extraction_rules})
messages.append({'role': 'user', 'content': user_command})       

isCodeGenerated=False

while(isCodeGenerated==False): 
    print("Generating code...")    
    response = generate_code()    
    if (response.function_call):
        load_info(response)
    else:
        isCodeGenerated=True
        print("Code generation completed!")        
        print(response.content)

Generating code...



Model wants to call a function 'rag_search_library'...
RAG Query: Find library for cut material vertical for SE-01 machine

Function 'rag_search_library' returned: The library package 'machine_cut_v' contains functions related to material cutting.\nHere's the list of available functions:\n# Cut the material vertically from the point (beginX,beginY) to the point (endX,endY)\nverticalCut(beginX, beginY, endX, endY)\n# Cut the material vertically from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nverticalCutZZ(beginX, beginY, endX, endY)The library package 'machine_cut_h' contains functions related to material cutting.\nHere's the list of available functions:\n# Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY)\nHCut(beginX, beginY, endX, endY)\n# Cut the material horizontally from the point (beginX,beginY) to the point (endX,endY) in a zigzag pattern\nHZZCut(beginX, beginY, endX, endY)The library package 'machine_solder_v' co